    
  - **Scaling:**  
    - Each selected metric is scaled by dividing by its maximum value.  
    - This normalizes all values to a range between 0 and 1, making them comparable across different scales.

- **Super-Efficiency DEA Model:**
  - **DEA (Data Envelopment Analysis) Concept:**  
    - Evaluates the relative efficiency of banks using multiple inputs and outputs.  
    - In standard DEA, each bank can use its own data as a benchmark, often resulting in a score of 1.
    
  - **Super-Efficiency Approach:**  
    - To better discriminate between banks, the model removes the bank under evaluation from the reference set.
    - For each bank (DMU), the code creates a reference set by deleting its data from the input and output matrices.
    - The optimization problem (using CVXPY) then minimizes a variable \( \theta \) subject to:
      - **Input Constraints:**  
        Weighted sum of the reference banks’ inputs must be ≤ \( \theta \) times the bank’s own inputs.
      - **Output Constraints:**  
        Weighted sum of the reference banks’ outputs must be ≥ the bank’s own outputs.
      - **Convexity Constraint:**  
        The weights (lambdas) must sum to 1.
    - A solution where \( \theta > 1 \) indicates that the bank is “super-efficient” compared to its peers.

https://www.sciencedirect.com/science/article/abs/pii/S0377221713003111

**DEA (Data Envelopment Analysis):**  
DEA is a method used to evaluate the efficiency of similar units, like businesses or hospitals, by comparing multiple inputs (resources used) and outputs (services or products produced) simultaneously. It constructs an "efficient frontier" from the best-performing units and then assigns efficiency scores (typically between 0 and 1) to all units based on how well they transform inputs into outputs, without assuming any predetermined relationship among them.

**Super-Efficiency DEA:**  
Super-Efficiency DEA builds upon standard DEA by further differentiating among those units that are already considered efficient. In this approach, when calculating the efficiency of a top-performing unit, that unit is temporarily excluded from the comparison set, allowing it to achieve a score greater than 1 if it performs exceptionally well. This adjustment provides a finer ranking among the best performers, highlighting degrees of excellence beyond the simple efficient/inefficient classification.

In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

In [15]:

def get_most_recent_value(df: pd.DataFrame, key: str, fallback_df: pd.DataFrame = None) -> float:
    try:
        v = df.loc[key].iloc[0]
        if pd.isna(v) and fallback_df is not None:
            return fallback_df.loc[key].iloc[0]
        return v
    except Exception:
        if fallback_df is not None:
            try:
                return fallback_df.loc[key].iloc[0]
            except Exception:
                return np.nan
        return np.nan

def get_total_liabilities(bs: pd.DataFrame, bs_q: pd.DataFrame = None) -> float:
    if bs is None:
        return np.nan
    if "Total Liab" in bs.index:
        return get_most_recent_value(bs, "Total Liab", fallback_df=bs_q)
    if "Total Liabilities" in bs.index:
        return get_most_recent_value(bs, "Total Liabilities", fallback_df=bs_q)
    if "Total Stockholder Equity" in bs.index:
        eq = get_most_recent_value(bs, "Total Stockholder Equity", fallback_df=bs_q)
        ta = get_most_recent_value(bs, "Total Assets",            fallback_df=bs_q)
        if pd.notna(eq) and pd.notna(ta):
            return ta - eq
    return np.nan

# 1) Universe of tickers
tickers = [
    'WFC','ALLY','AXP','AMP','ASB','BAC','BK','BOKF','COF','SCHW',
    'C','CFG','COLB','CMA','CFR','DFS','EWBC','FNB','FITB','FCNCA',
    'FHN','FLG','GS','HSBC','HBAN','DE','JPM','KEY','MTB','MS',
    'NTRS','ONB','PNFP','PNC','BPOP','PB','RJF','RF','SC','SSB',
    'STT','SYF','SNV','TFC','USB','UMBF','VLY','WBS','WAL','WTFC'
]

# 2) Fetch & compute raw metrics
records = []
for t in tickers:
    tk      = yf.Ticker(t)
    bs, bs_q   = tk.balance_sheet, tk.quarterly_balance_sheet
    fin, fin_q = tk.financials,   tk.quarterly_financials
    info       = tk.info

    ta  = get_most_recent_value(bs, "Total Assets",           fallback_df=bs_q)
    tl  = get_total_liabilities(bs, bs_q)
    ni  = get_most_recent_value(fin, "Net Income",            fallback_df=fin_q)
    rev = get_most_recent_value(fin, "Total Revenue",         fallback_df=fin_q)
    op  = get_most_recent_value(fin, "Operating Income",      fallback_df=fin_q) or ni

    rec = {'Ticker': t}
    rec['totalAssets']      = ta
    rec['debtToEquity']     = (tl/(ta-tl)
                               if pd.notna(ta) and pd.notna(tl) and (ta-tl)!=0
                               else np.nan)
    rec['priceToBook']      = info.get('priceToBook', np.nan)
    rec['returnOnAssets']   = (ni/ta
                               if pd.notna(ni) and pd.notna(ta) and ta!=0
                               else np.nan)
    rec['returnOnEquity']   = (ni/(ta-tl)
                               if pd.notna(ni) and pd.notna(ta) and pd.notna(tl) and (ta-tl)!=0
                               else np.nan)
    rec['profitMargins']    = (ni/rev
                               if pd.notna(ni) and pd.notna(rev) and rev!=0
                               else np.nan)
    rec['operatingMargins'] = (op/rev
                               if pd.notna(op) and pd.notna(rev) and rev!=0
                               else np.nan)

    records.append(rec)

df = pd.DataFrame(records).set_index('Ticker')

# 3) Fill NaNs with column max (no chained assignment)
for col in df.columns:
    series = pd.to_numeric(df[col], errors='coerce')
    max_val = series.max(skipna=True)
    df[col] = series.fillna(max_val)

# drop any tickers with all-NaN metrics (e.g. if 'C' or 'FLG' had no data)
df.dropna(how='all', inplace=True)

# 4) DEA inputs & outputs
inputs  = ['debtToEquity', 'priceToBook']
outputs = ['totalAssets','returnOnAssets','returnOnEquity','profitMargins','operatingMargins']

inputs  = [c for c in inputs  if c in df and df[c].nunique()>1]
outputs = [c for c in outputs if c in df and df[c].nunique()>1]

data = df[inputs + outputs].copy()
for c in data.columns:
    mx = data[c].max()
    if mx>0:
        data[c] /= mx

# 5) Output‑oriented super‑efficiency DEA
X = data[inputs].values
Y = data[outputs].values
n, m = X.shape
_, s = Y.shape

phi_vals = {}
for i in range(n):
    X_ref = np.delete(X, i, axis=0)
    Y_ref = np.delete(Y, i, axis=0)

    φ   = cp.Variable(nonneg=True)
    lam = cp.Variable(n-1, nonneg=True)
    cons = []
    for j in range(m):
        cons.append(cp.sum(cp.multiply(lam, X_ref[:, j])) <= X[i, j])
    for k in range(s):
        cons.append(cp.sum(cp.multiply(lam, Y_ref[:, k])) >= φ * Y[i, k])
    cons.append(cp.sum(lam) == 1)

    prob = cp.Problem(cp.Maximize(φ), cons)
    prob.solve(solver=cp.SCS, verbose=False)
    phi_vals[data.index[i]] = φ.value

results = pd.DataFrame.from_dict(phi_vals, orient='index', columns=['phi'])
max_phi = results['phi'].replace([np.inf, None], np.nan).max(skipna=True)
results['phi'] = results['phi'].apply(lambda x: max_phi if x is None or np.isinf(x) else x)

# 6) Rescale DEA φ → base_score (1–5)
phi = results['phi']
results['base_score'] = 1 + 4*(phi - phi.min())/(phi.max() - phi.min())

# 7) Size bias blend
size_score = 1 + 4*(df['totalAssets']/df['totalAssets'].max())
weight = 0.5
results['raw'] = (1-weight)*results['base_score'] + weight*size_score
min_r, max_r = results['raw'].min(), results['raw'].max()
results['final_score_1_5'] = 1 + 4*(results['raw']-min_r)/(max_r-min_r)
results['final_score_1_5'] = results['final_score_1_5'].round(2)

print(results[['phi','base_score','raw','final_score_1_5']].sort_values('final_score_1_5', ascending=False))

             phi  base_score       raw  final_score_1_5
KEY    16.305998    5.000000  3.093518             5.00
JPM     0.837474    1.071719  3.035859             4.89
SC      0.555064    1.000000  3.000000             4.82
BAC     0.838426    1.071961  2.665593             4.16
HSBC    1.177351    1.158032  2.586479             4.00
WFC     1.303454    1.190056  2.059272             2.97
GS      1.383995    1.210510  1.942652             2.74
MS      1.620230    1.270502  1.742360             2.34
USB     1.573010    1.258511  1.468176             1.81
ALLY    3.526384    1.754576  1.473139             1.81
COF     2.159177    1.407370  1.448584             1.77
PNC     1.347234    1.201174  1.380409             1.63
SCHW    1.498483    1.239584  1.359545             1.59
STT     1.866742    1.333105  1.343048             1.56
BK      1.593815    1.263794  1.339783             1.55
TFC     0.993642    1.111378  1.321090             1.52
AMP     2.305294    1.444476  1.312876          

In [11]:
output_path = "/Users/arunnimmagadda/Downloads/DEAscoring.csv"
results.to_csv(output_path, index=True)
print(f"\nSaved composite scores to {output_path}")


Saved composite scores to /Users/arunnimmagadda/Downloads/DEAscoring.csv
